In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import math
from train import ModelTrainer
from collection import Collection

In [2]:
try:
    collection
except NameError:
    collection = Collection()

# Part 1: Setup Helpers (manual)

## Persist current collection

In [4]:
collection.to_file('dbs/tune.json')

## Restore from file (careful)

In the next box you can restore a list of runs from a json file

In [ ]:
collection.import_file('dbs/tune.json', force=True)

## Reset model cache

This needs to be done when a new query should be done e.g. anbieter has changed

Maybe implement cache invalidation in ModelTrainer

In [ ]:
trainer.resetSQLData()

## set config

In [ ]:
trainer.config = config

## Set attributes in trainer

# Part 2: Config

In [3]:
# Prepare Attributes
def cleanData(df, filters):
#    if 'beschaffungsstelle_plz' in filters:
#        df[['beschaffungsstelle_plz']] = df[['beschaffungsstelle_plz']].applymap(ModelTrainer.tonumeric)
    if 'gatt_wto' in filters:
        df[['gatt_wto']] = df[['gatt_wto']].applymap(ModelTrainer.unifyYesNo)
    if 'anzahl_angebote' in filters:
        df[['anzahl_angebote']] = df[['anzahl_angebote']].applymap(ModelTrainer.tonumeric)
    if 'teilangebote' in filters:
        df[['teilangebote']] = df[['teilangebote']].applymap(ModelTrainer.unifyYesNo)
    if 'lose' in filters:
        df[['lose']] = df[['lose']].applymap(ModelTrainer.unifyYesNo)
    if 'varianten' in filters:
        df[['varianten']] = df[['varianten']].applymap(ModelTrainer.unifyYesNo)
    if 'auftragsart_art' in filters:
        auftrags_art_df = pd.get_dummies(df['auftragsart_art'], prefix='aftrgsrt',dummy_na=True)
        df = pd.concat([df,auftrags_art_df],axis=1).drop(['auftragsart_art'],axis=1)
    if 'sprache' in filters:
        sprache_df = pd.get_dummies(df['sprache'], prefix='lang',dummy_na=True)
        df = pd.concat([df,sprache_df],axis=1).drop(['sprache'],axis=1)
    if 'auftragsart' in filters:
        auftragsart_df = pd.get_dummies(df['auftragsart'], prefix='auftr',dummy_na=True)
        df = pd.concat([df,auftragsart_df],axis=1).drop(['auftragsart'],axis=1)
    if 'beschaffungsstelle_plz' in filters:
        plz_df = pd.get_dummies(df['beschaffungsstelle_plz'], prefix='beschaffung_plz',dummy_na=True)
        df = pd.concat([df,plz_df],axis=1).drop(['beschaffungsstelle_plz'],axis=1)
    return df

In [4]:
select_anbieter = (
    # "anbieter.anbieter_id, "
    "anbieter.institution as anbieter_institution, "
    "cpv_dokument.cpv_nummer as anbieter_cpv, "
    "ausschreibung.meldungsnummer"
)
# anbieter_CPV are all the CPVs the Anbieter ever won a procurement for. So all the CPVs they are interested in. 
select_ausschreibung = (
    # "anbieter.anbieter_id, "
    "auftraggeber.institution as beschaffungsstelle_institution, "
    "auftraggeber.beschaffungsstelle_plz, "
    "ausschreibung.gatt_wto, "
    "ausschreibung.sprache, "
    "ausschreibung.auftragsart, "
    "ausschreibung.auftragsart_art, "
    "ausschreibung.lose, "
    "ausschreibung.teilangebote, "
    "ausschreibung.varianten, "
    # "ausschreibung.projekt_id, "
    # "ausschreibung.titel, "
    "ausschreibung.bietergemeinschaft, "
    "cpv_dokument.cpv_nummer as ausschreibung_cpv, "
    "ausschreibung.meldungsnummer as meldungsnummer2"
)

In [5]:
config = {
    # ratio that the positive and negative responses have to each other
    'positive_to_negative_ratio': 0.5,
    # Percentage of training set that is used for testing (Recommendation of at least 25%)
    'test_size': 0.25,
    'runs': 100,
    #'enabled_algorithms': ['random_forest'],
    'enabled_algorithms': ['random_forest', 'decision_tree', 'gradient_boost'],
    'random_forest': {
        # Tune Random Forest Parameter
        'n_estimators': 400,
        'max_features': 'sqrt',
        'max_depth': None,
        'min_samples_split': 2
    },
    'decision_tree': {
        'max_depth': 6,
        'max_features': 'sqrt'
    },
    'gradient_boost': {
        'n_estimators': 100,
        'learning_rate': 0.1,
        'max_depth': 6,
        'max_features': 'sqrt'
    }
}

In [6]:
try:
    trainer
except NameError:
    trainer = ModelTrainer(select_anbieter, select_ausschreibung, '', config, cleanData, [])

In [7]:
#attributes = ['ausschreibung_cpv', 'gatt_wto']
attributes = ['ausschreibung_cpv', 'auftragsart_art','beschaffungsstelle_plz','gatt_wto','lose','teilangebote', 'varianten','sprache']
#attributes = ['auftragsart_art','beschaffungsstelle_plz','gatt_wto','lose','teilangebote', 'varianten','sprache',]
trainer.attributes = attributes

In [7]:
# Choose a bidder to train a model for (number of positive marked after the name)

# === THESIS ===

#anbieter = 'Alpiq AG' #430
#anbieter = 'Swisscom' #302
anbieter = 'Kummler + Matter AG' #160
#anbieter = 'Siemens AG' #532

#anbieter = 'G. Baumgartner AG' #65
#anbieter = 'ELCA Informatik AG' #125
#anbieter = 'Thermo Fisher Scientific (Schweiz) AG' #160
#anbieter = 'Arnold AG' #82

#anbieter = 'Riget AG' #21
#anbieter = 'isolutions AG' #16
#anbieter = 'CSI Consulting AG' #21
#anbieter = 'Aebi & Co. AG Maschinenfabrik' #15

#anbieter = 'DB Schenker AG' #6
#anbieter = 'IT-Logix AG' #12
#anbieter = 'AVS Syteme AG' #14
#anbieter = 'Sajet SA' #7

# === TESTING ===

#anbieter = 'Marti AG' #456
#anbieter = 'Axpo AG' #40
#anbieter = 'Hewlett-Packard' #90
#anbieter = 'BG Ingénieurs Conseils' SA #116
#anbieter = 'Pricewaterhousecoopers' #42
#anbieter = 'Helbling Beratung + Bauplanung AG' #20
#anbieter = 'Ofrex SA' #52
#anbieter = 'PENTAG Informatik AG' #10
#anbieter = 'Wicki Forst AG' #12
#anbieter = 'T-Systems Schweiz' #18
#anbieter = 'Bafilco AG' #20
#anbieter = '4Video-Production GmbH' #3
#anbieter = 'Widmer Ingenieure AG' #6
#anbieter = 'hmb partners AG' #2
#anbieter = 'Planmeca' #4
#anbieter = 'K & M Installationen AG' #4

trainer.anbieter = anbieter

# Whatever

In [8]:
trainer.queryData()

INFO:train:sql done


(     anbieter_institution  anbieter_cpv  meldungsnummer  \
 0     Kummler + Matter AG      45234160          409325   
 1     Kummler + Matter AG      71000000          439377   
 2     Kummler + Matter AG      45234160          441553   
 3     Kummler + Matter AG      31321100          681471   
 4     Kummler + Matter AG      45231400          703911   
 5     Kummler + Matter AG      51000000          729245   
 6     Kummler + Matter AG      51000000          729245   
 7     Kummler + Matter AG      51000000          729245   
 8     Kummler + Matter AG      51000000          729245   
 9     Kummler + Matter AG      51100000          729245   
 10    Kummler + Matter AG      51100000          729245   
 11    Kummler + Matter AG      51100000          729245   
 12    Kummler + Matter AG      51100000          729245   
 13    Kummler + Matter AG       9330000          729245   
 14    Kummler + Matter AG       9330000          729245   
 15    Kummler + Matter AG       9330000

In [9]:
trainer.positives

,anbieter_institution,anbieter_cpv,meldungsnummer,beschaffungsstelle_institution,beschaffungsstelle_plz,gatt_wto,sprache,auftragsart,auftragsart_art,lose,teilangebote,varianten,bietergemeinschaft,ausschreibung_cpv,meldungsnummer2
0,Kummler + Matter AG,45234160,409325,None,1212,YES,FR,Bauauftrag,EXECUTION,nein,NO,NO,Admises selon l'art. 34 du règlement cantonal....,45234160,409325
1,Kummler + Matter AG,71000000,439377,SBB - Schweizerische Bundesbahnen,1001,NO,FR,Dienstleistungsauftrag,CPC12,nein,NO,YES,admises,71000000,439377
2,Kummler + Matter AG,45234160,441553,Stadt St. Gallen,9001,NO,DE,Bauauftrag,EXECUTION,nein,NO,NO,,45234160,441553
3,Kummler + Matter AG,31321100,681471,Romande Energie,1110,YES,FR,Bauauftrag,EXECUTION,nein,NO,NO,,31321100,681471
4,Kummler + Matter AG,45231400,703911,NOK - Nordostschweizerische Kraftwerke AG,5401,YES,DE,Bauauftrag,EXECUTION,6,NO,YES,Zugelassen mit federführender Unternehmumg,45231400,703911
5,Kummler + Matter AG,51000000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,51000000,729245
6,Kummler + Matter AG,51000000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,51100000,729245
7,Kummler + Matter AG,51000000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,9330000,729245
8,Kummler + Matter AG,51000000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,9332000,729245
9,Kummler + Matter AG,51100000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,51000000,729245


In [31]:
df = trainer.positives.copy()
df = df.drop_duplicates().reset_index(drop=True)
df

,anbieter_institution,anbieter_cpv,meldungsnummer,beschaffungsstelle_institution,beschaffungsstelle_plz,gatt_wto,sprache,auftragsart,auftragsart_art,lose,teilangebote,varianten,bietergemeinschaft,ausschreibung_cpv,meldungsnummer2
0,Kummler + Matter AG,45234160,409325,None,1212,YES,FR,Bauauftrag,EXECUTION,nein,NO,NO,Admises selon l'art. 34 du règlement cantonal....,45234160,409325
1,Kummler + Matter AG,71000000,439377,SBB - Schweizerische Bundesbahnen,1001,NO,FR,Dienstleistungsauftrag,CPC12,nein,NO,YES,admises,71000000,439377
2,Kummler + Matter AG,45234160,441553,Stadt St. Gallen,9001,NO,DE,Bauauftrag,EXECUTION,nein,NO,NO,,45234160,441553
3,Kummler + Matter AG,31321100,681471,Romande Energie,1110,YES,FR,Bauauftrag,EXECUTION,nein,NO,NO,,31321100,681471
4,Kummler + Matter AG,45231400,703911,NOK - Nordostschweizerische Kraftwerke AG,5401,YES,DE,Bauauftrag,EXECUTION,6,NO,YES,Zugelassen mit federführender Unternehmumg,45231400,703911
5,Kummler + Matter AG,51000000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,51000000,729245
6,Kummler + Matter AG,51000000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,51100000,729245
7,Kummler + Matter AG,51000000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,9330000,729245
8,Kummler + Matter AG,51000000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,9332000,729245
9,Kummler + Matter AG,51100000,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,51000000,729245


In [38]:
df_without_cpv = df.drop(['ausschreibung_cpv', 'anbieter_cpv'], axis=1)
df_without_cpv = df_without_cpv.drop_duplicates()
df_without_cpv

,anbieter_institution,meldungsnummer,beschaffungsstelle_institution,beschaffungsstelle_plz,gatt_wto,sprache,auftragsart,auftragsart_art,lose,teilangebote,varianten,bietergemeinschaft,meldungsnummer2,cpv_division,cpv_group,cpv_class,cpv_category
0,Kummler + Matter AG,409325,None,1212,YES,FR,Bauauftrag,EXECUTION,nein,NO,NO,Admises selon l'art. 34 du règlement cantonal....,409325,45,452,4523,45234
1,Kummler + Matter AG,439377,SBB - Schweizerische Bundesbahnen,1001,NO,FR,Dienstleistungsauftrag,CPC12,nein,NO,YES,admises,439377,71,710,7100,71000
2,Kummler + Matter AG,441553,Stadt St. Gallen,9001,NO,DE,Bauauftrag,EXECUTION,nein,NO,NO,,441553,45,452,4523,45234
3,Kummler + Matter AG,681471,Romande Energie,1110,YES,FR,Bauauftrag,EXECUTION,nein,NO,NO,,681471,31,313,3132,31321
4,Kummler + Matter AG,703911,NOK - Nordostschweizerische Kraftwerke AG,5401,YES,DE,Bauauftrag,EXECUTION,6,NO,YES,Zugelassen mit federführender Unternehmumg,703911,45,452,4523,45231
5,Kummler + Matter AG,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,729245,51,510,5100,51000
6,Kummler + Matter AG,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,729245,51,511,5110,51100
7,Kummler + Matter AG,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,729245,9,93,933,9330
8,Kummler + Matter AG,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,YES,,729245,9,93,933,9332
21,Kummler + Matter AG,730715,ASTRA - Bundesamt für Strassen,6500,YES,IT,Bauauftrag,EXECUTION,nein,NO,NO,"Ammessi, a condizione che un offerente si assu...",730715,31,317,3171,31711


In [39]:
split = {
    'division': lambda x: math.floor(x/1000000),
    'group': lambda x: math.floor(x/100000),
    'class': lambda x: math.floor(x/10000),
    'category': lambda x: math.floor(x/1000)
}

for key, applyFun in split.items():
    df['cpv_' + key ] = df['ausschreibung_cpv'].apply(applyFun)

df[['cpv_' + key for key, fun in split.items()]]

,cpv_division,cpv_group,cpv_class,cpv_category
0,45,452,4523,45234
1,71,710,7100,71000
2,45,452,4523,45234
3,31,313,3132,31321
4,45,452,4523,45231
5,51,510,5100,51000
6,51,511,5110,51100
7,9,93,933,9330
8,9,93,933,9332
9,51,510,5100,51000


In [40]:
tmpdf = {}
for key in split.keys():
    key = 'cpv_' + key
    tmpdf[key] = df[['meldungsnummer']].join(pd.get_dummies(df[key], prefix=key)).groupby('meldungsnummer').max()
    
encoded_df = pd.concat([ tmpdf['cpv_'+ key] for key in split.keys()], axis=1)
df.drop(['cpv_' + key for key, fun in split.items()], axis=1)
encoded_df

,cpv_division_9,cpv_division_31,cpv_division_32,cpv_division_34,cpv_division_35,cpv_division_38,cpv_division_42,cpv_division_44,cpv_division_45,cpv_division_48,...,cpv_category_71320,cpv_category_71322,cpv_category_71323,cpv_category_71340,cpv_category_71356,cpv_category_71541,cpv_category_77231,cpv_category_79611,cpv_category_79620,cpv_category_79623
meldungsnummer,,,,,,,,,,,,,,,,,,,,,
409325,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
439377,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
441553,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
681471,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
703911,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
729245,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
730715,0,1,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
736937,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
737959,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
join_df = df_without_cpv.join(encoded_df, on='meldungsnummer')
join_df

,anbieter_institution,meldungsnummer,beschaffungsstelle_institution,beschaffungsstelle_plz,gatt_wto,sprache,auftragsart,auftragsart_art,lose,teilangebote,...,cpv_category_71320,cpv_category_71322,cpv_category_71323,cpv_category_71340,cpv_category_71356,cpv_category_71541,cpv_category_77231,cpv_category_79611,cpv_category_79620,cpv_category_79623
0,Kummler + Matter AG,409325,None,1212,YES,FR,Bauauftrag,EXECUTION,nein,NO,...,0,0,0,0,0,0,0,0,0,0
1,Kummler + Matter AG,439377,SBB - Schweizerische Bundesbahnen,1001,NO,FR,Dienstleistungsauftrag,CPC12,nein,NO,...,0,0,0,0,0,0,0,0,0,0
2,Kummler + Matter AG,441553,Stadt St. Gallen,9001,NO,DE,Bauauftrag,EXECUTION,nein,NO,...,0,0,0,0,0,0,0,0,0,0
3,Kummler + Matter AG,681471,Romande Energie,1110,YES,FR,Bauauftrag,EXECUTION,nein,NO,...,0,0,0,0,0,0,0,0,0,0
4,Kummler + Matter AG,703911,NOK - Nordostschweizerische Kraftwerke AG,5401,YES,DE,Bauauftrag,EXECUTION,6,NO,...,0,0,0,0,0,0,0,0,0,0
5,Kummler + Matter AG,729245,Kanton Basel-Landschaft,4410,NO,DE,Bauauftrag,EXECUTION,nein,NO,...,0,0,0,0,0,0,0,0,0,0
21,Kummler + Matter AG,730715,ASTRA - Bundesamt für Strassen,6500,YES,IT,Bauauftrag,EXECUTION,nein,NO,...,0,0,0,0,0,0,0,0,0,0
121,Kummler + Matter AG,736937,ASTRA - Bundesamt für Strassen,6500,YES,IT,Bauauftrag,EXECUTION,nein,NO,...,0,0,0,0,0,0,0,0,0,0
122,Kummler + Matter AG,737959,Verkehrsbetriebe Zürich VBZ,8048,YES,DE,Bauauftrag,EXECUTION,nein,NO,...,0,0,0,0,0,0,0,0,0,0
126,Kummler + Matter AG,739423,ASTRA - Bundesamt für Strassen,6500,YES,IT,Bauauftrag,EXECUTION,nein,NO,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df.groupby('meldungsnummer').nunique()

,anbieter_institution,anbieter_cpv,meldungsnummer,beschaffungsstelle_institution,beschaffungsstelle_plz,gatt_wto,sprache,auftragsart,auftragsart_art,lose,teilangebote,varianten,projekt_id,bietergemeinschaft,ausschreibung_cpv,meldungsnummer2,cpv_division,cpv_group,cpv_class,cpv_category
meldungsnummer,,,,,,,,,,,,,,,,,,,,
380721,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
384773,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
390515,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
393571,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
404931,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,2
406415,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
408753,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
412181,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
412459,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [27]:
df[df['meldungsnummer']==1030745]

,anbieter_institution,anbieter_cpv,meldungsnummer,beschaffungsstelle_institution,beschaffungsstelle_plz,gatt_wto,sprache,auftragsart,auftragsart_art,lose,teilangebote,varianten,projekt_id,bietergemeinschaft,ausschreibung_cpv,meldungsnummer2,cpv_division,cpv_group,cpv_class,cpv_category
1078,Siemens AG,31200000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31200000,1030745,31,312,3120,31200
1079,Siemens AG,31210000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31200000,1030745,31,312,3120,31200
1080,Siemens AG,31230000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31230000,1030745,31,312,3123,31230
1081,Siemens AG,42961000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31200000,1030745,31,312,3120,31200
1082,Siemens AG,42961000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31230000,1030745,31,312,3123,31230
1083,Siemens AG,31230000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31200000,1030745,31,312,3120,31200
1084,Siemens AG,31200000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31230000,1030745,31,312,3123,31230
1085,Siemens AG,31210000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31230000,1030745,31,312,3123,31230
1086,Siemens AG,42961000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31210000,1030745,31,312,3121,31210
1087,Siemens AG,31200000,1030745,Swissgrid,5000,YES,FR,Lieferauftrag,WORKS.CONTRACT,nein,NO,NO,174231,ne sont pas admise,31210000,1030745,31,312,3121,31210


In [53]:
df.duplicated(['meldungsnummer'])

meldungsnummer
384379     False
391049      True
396867      True
402827      True
405665      True
410729      True
413079      True
419461      True
440703      True
442419      True
445071      True
447401      True
447419      True
449413      True
451097      True
452899      True
452903      True
459965      True
463151      True
465095      True
472099      True
474127      True
487431      True
488241      True
493849      True
496357      True
508051      True
512041      True
526897      True
546003      True
           ...  
970317      True
970503      True
970735      True
971433      True
972205      True
972233      True
974111      True
974311      True
976193      True
976731      True
976745      True
977343      True
978527      True
982993      True
986513      True
987297      True
988245      True
988875      True
989507      True
995349      True
1001121     True
1005489     True
1005491     True
1012179     True
1013617     True
1014599     True
1016291     True